In [26]:
# compare route outputs from different levels of complexity
import os, datetime
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import multiprocessing as mp
from tqdm import tqdm 

# ========== User settings ==================================
root_dir='/glade/u/home/hongli/scratch/2020_06_02HRUcomplexity/model'
basinName='06279940'

# hru inputs
level_num=0 #3
label_basename='hru_lev'

# hru shapefile
case = 'shoshone'
discretize_dir = '/glade/u/home/hongli/scratch/2020_06_02HRUcomplexity/discretize'
case_dir = os.path.join(discretize_dir,case)
sub_shp_prj = os.path.join(discretize_dir, case, 'subbasin_prj.shp')
stream_clip = os.path.join(discretize_dir, case, 'stream.shp')
wgs_crs = 'epsg:4326'

# plot inputs
time_format = '%Y-%m-%d'
plot_date_start = '2007-10-01' #'2007-10-01'
plot_date_end = '2008-09-30'
plot_date_start_obj = datetime.datetime.strptime(plot_date_start, time_format)
plot_date_end_obj = datetime.datetime.strptime(plot_date_end, time_format)
plot_date_range = pd.date_range(start=plot_date_start,end=plot_date_end,freq='M').to_pydatetime().tolist()

# temperature, precipitation, radiation, SWE, SM, ET, totalrunoff
var_names_summa = ['spectralIncomingDirect_mean', 'spectralIncomingDiffuse_mean','pptrate_mean', 'airtemp_mean', 'scalarCanopyWat', 'scalarSWE', 'scalarAquiferStorage', 
                  'scalarTotalSoilWat','scalarSurfaceRunoff_mean',  'scalarAquiferBaseflow_mean', 'scalarTotalET_mean', 'scalarTotalRunoff_mean'] 
var_units_summa = ['$(W/m^2)$','$(W/m^2)$','kg m-2 s-1', 'K', 'kg m-2', 'kg m-2','m', 'kg m-2', 'm s-1', 'm s-1', 'kg m-2 s-1', 'm s-1']  

var_names_plot = ['Direct Shortwave Rad','Diffuse Shortwave Rad','Precip Rate', 'Temprature', 'Canopy Water', 'SWE', 'Aquifer Storage', 
                 'Total Soil Water','Surface Runoff', 'Aquifer Baseflow', 'Total ET', 'Total Runoff']  
var_units_plot = ['$(W/m^2)$','$(W/m^2)$','(mm/d)', '$(^\circ$C)', '(mm)', '(mm)','(mm)', '(mm)', '(mm/d)', '(mm/d)', '(mm/d)', '(mm/d)'] 

output_dir=os.path.join(root_dir,basinName,'analysis/9_plot_summa_output_space')
for level_id in range(level_num+1):
    label=label_basename+str(level_id)
    output_subdir=os.path.join(output_dir,label)
    if not os.path.exists(output_subdir):
        os.makedirs(output_subdir)

# ========== end User settings ==================================

# ##============== PART 1. Read data ==============
# # --- read summa input & output
# print('read')
# data_dic = {}
# var_units_update = []
# for level_id in range(level_num+1):
#     label=label_basename+str(level_id)
#     data_lev_dic = {}
    
#     # (2) read other variables from summa output
#     q_ncfile = os.path.join(root_dir,basinName,'output',label,'wbout_day.nc')
#     ds = xr.open_dataset(q_ncfile) 
#     time = ds['time'].values[:]
#     time = pd.to_datetime(time)
    
#     for j in range(len(var_names_summa)): # variable
#         var_name_summa = var_names_summa[j]
#         var_unit = var_units_summa[var_names_summa.index(var_name_summa)]
        
#         var_value = ds[var_name_summa].values[:] # (time, hru). or (time, spectral_bands, hru) for radiation
       
#         # sum up the visible and near-infra-red spectrums
#         if var_name_summa in ['spectralIncomingDirect_mean', 'spectralIncomingDiffuse_mean']:
#             if np.count_nonzero(var_value) == 0:
#                 print('***** %s contains only zeros *****'%(var_name_summa))
#             var_value = np.nansum(var_value,axis=1)
            
#         # convert unit
#         if var_unit == 'kg m-2 s-1': # flux to mm/day
#             var_value = var_value*86400 
#         elif var_unit == 'kg m-2': # unit water storage to depth
#             var_value = var_value
#         elif var_unit == 'm s-1': # rate to mm/d
#             var_value = var_value*1000*86400 
#         elif var_unit == 'K': # temperature to C
#             var_value = var_value - 273.15 
#         elif var_unit == 'm': # to mm
#             var_value = var_value*1000 
        
#         df = pd.DataFrame(var_value, index=time)
    
#         # calculate monthly mean 
#         df_avg = df.resample('M').mean()
#         data_lev_dic[var_name_summa] = df_avg   

#     # save to the big dic
#     data_dic[label]=data_lev_dic
    
# # --- read GRU shapefile
# gru_gpd = gpd.read_file(sub_shp_prj)
# gru_gpd_prj = gru_gpd.to_crs(wgs_crs)

# # --- read stream shapefile
# stream_gpd = gpd.read_file(stream_clip)
# stream_gpd_prj = stream_gpd.to_crs(wgs_crs)

# # --- read HRU shapefile
# hru_gpd_dic = {}
# for i in range(level_num+1): # hru level
#     label=label_basename+str(i) 

#     # read HRU shapefile 
#     hru_elmn_str = label+'_elmn'     
#     hru_vector_elmn = os.path.join(case_dir, hru_elmn_str+'.shp')          
#     hru_gpd = gpd.read_file(hru_vector_elmn)
#     hru_gpd_prj = hru_gpd.to_crs(wgs_crs)
#     hru_gpd_dic[label]=hru_gpd_prj

# # ============== PART 2. Identify vmin and vmax ==============
# print('vmin/vamx')
# vmin_vmax_dic = {}
# for i in range(level_num+1): # hru level
#     label=label_basename+str(i) 
#     vmin_vmax_lev_dic = {}
    
#     for j in range(len(var_names_summa)): # variable
#         var_name_summa = var_names_summa[j]

#         # extract usefule variable value
#         df = data_dic[label][var_name_summa]
#         df_cut = df.truncate(before=plot_date_start_obj, after=plot_date_end_obj)
#         data = df_cut.to_numpy()
        
#         vmin,vmax = np.nanmin(data),np.nanmax(data)
#         vmin_vmax_lev_dic[var_name_summa] = [vmin,vmax]
    
#     # save to the big dic
#     vmin_vmax_dic[label]=vmin_vmax_lev_dic    

# ============== PART 3. Daily plot ==============
print('plot')
fontsize = 'medium'
for level_i in range(level_num+1): # hru level
    label=label_basename+str(level_i) 
    hru_gpd_prj = hru_gpd_dic[label]
    output_subdir=os.path.join(output_dir,label)
    
    pool = mp.Pool(mp.cpu_count())    
    for t_obj in tqdm(plot_date_range[0:1]):
        t_obj_str = t_obj.strftime('%Y-%m')

        nrow, ncol = 4,3 # variables
        fig, ax = plt.subplots(nrow, ncol,figsize=[7.08*1.5*1.4, 7.08*1.2*1.5])#, constrained_layout=True)
        fig.subplots_adjust(left=0.01, bottom=0.05, right=0.98, top=0.94, wspace=0.0, hspace=0.3)
        fig.suptitle(label+' '+t_obj_str, fontsize='large',weight='semibold')

        for i in range(nrow): 
            for j in range(ncol): 

                # identify variable and unit 
                k = i*ncol+j # variable index                
                var_name_summa = var_names_summa[k]
                var_name_plot = var_names_plot[k]
                var_unit_plot = var_units_plot[k]  
                
                vmin,vmax = vmin_vmax_dic[label][var_name_summa]
                if vmin==vmax and vmin==0:
                    vmin,vmax = -10,10

                # extract usefule variable value
                df = data_dic[label][var_name_summa] # dataframe (time, hru)
                df_cut = df.truncate(before=t_obj, after=t_obj)  
                data = df_cut.to_numpy()
                data = data.reshape((np.shape(data)[1],np.shape(data)[0]))            

                # plot
                # (1) plot gru
                gru_gpd_prj.geometry.boundary.plot(color=None,edgecolor='k',linewidth=0.5,
                                                   ax=ax[i,j],label='GRU') 

    #             # (2) plot stream
    #             stream_gpd_prj.plot(color='b', linewidth=0.5, ax=ax[i,j], label='Stream')

                # (3) plot data
                # reference: https://geopandas.org/docs/user_guide/mapping.html
                # reference: https://www.martinalarcon.org/2018-12-31-d-geopandas/
                hru_gpd_prj[var_name_summa] = data
                divider = make_axes_locatable(ax[i,j])
                cax = divider.append_axes("right", size="5%", pad=-0.5)         
                hru_gpd_prj.plot(column=var_name_summa, ax=ax[i,j], cmap='jet',
                                 legend=True, cax=cax, vmin=vmin, vmax=vmax) #cmap='jet''Blues''terrain'

                # Set the fontsize for each colorbar tick label
                # manipulate the colorbar `cax`
                cax.set_title(var_unit_plot, fontsize=fontsize)
                for l in cax.yaxis.get_ticklabels():
                    l.set_fontsize(fontsize)

                # others
                title = var_name_plot
                ax[i,j].set_title(title,fontsize=fontsize,weight='semibold')
                ax[i,j].tick_params(axis='both', direction='out', labelsize=fontsize)

        ofile = t_obj.strftime('%Y%m')+'.png'
        plt.savefig(os.path.join(output_subdir,ofile), dpi=80)
        plt.close(fig)    
    pool.close()   

# ## Then convert daily plots to gif using command line.
# ## convert -delay 10 plot_daily/*.png test.gif
print('Done')

plot


100%|██████████| 1/1 [00:10<00:00, 10.70s/it]

Done


In [7]:
dd = data_dic['hru_lev0']['spectralIncomingDiffuse_mean']

In [10]:
vmin_vmax_dic[label]['spectralIncomingDiffuse_mean']

[0.0, 0.0]